In [2]:
from doc2text.reader import Reader
import cv2
import numpy as np
from PIL import Image
import pytesseract
import os
import warnings
import keras_ocr
import tensorflow

warnings.simplefilter('ignore')

In [37]:
class GetBoxes:
    def __init__(self) -> None:
        self.pipeline = keras_ocr.pipeline.Pipeline()
    
    @staticmethod
    def crop_rectangle(image: np.ndarray, box: np.ndarray) -> np.ndarray:
        """
        Вырезает прямоугольник из изображения по координатам четырех углов.

        Параметры:
        image (np.ndarray): Изображение, из которого нужно вырезать прямоугольник.
        box (np.ndarray): Массив координат четырех углов прямоугольника.

        Возвращает:
        np.ndarray: Вырезанный прямоугольник.
        """
        # Находим размер вырезаемого прямоугольника
        width = int(np.linalg.norm(box[0] - box[1]))
        height = int(np.linalg.norm(box[1] - box[2]))

        # Определяем целевые точки (прямоугольник с найденными размерами)
        dst_pts = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype="float32")

        # Вычисляем матрицу перспективного преобразования
        M = cv2.getPerspectiveTransform(box.astype("float32"), dst_pts)

        # Применяем преобразование и вырезаем прямоугольник
        cropped_image = cv2.warpPerspective(image, M, (width, height))

        return cropped_image


    def get_box(self, img_path: str, img_to='temp') -> list:
        if not os.path.exists(img_to):
            os.mkdir(img_to)
        image = keras_ocr.tools.read(img_path)
        predictions = self.pipeline.recognize([image])[0]
        boxes = [box for _, box in predictions]
        for i, box in enumerate(boxes):
            src = img_to + os.sep + f'{i}.jpg'
            img = self.crop_rectangle(image, box)
            Image.fromarray(img).save(src)
            

    


In [38]:
gtb = GetBoxes()

Looking for C:\Users\User\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\User\.keras-ocr\crnn_kurapan.h5


In [39]:
gtb.get_box(r'1\IMG-20240504-WA0070.jpg')

5/5 [==============================] - 7s 978ms/step


In [18]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
rd = Reader()

recognizer weights has loaded from c:\Users\User\Desktop\II\Tekst\train_RZHD_TrudovieKnizhki\doc2text\weights/ocr_transformer_4h2l_simple_conv_64x256.pt


In [11]:
line = rd.doc2text(r'Скриншот 17-05-2024 232603.jpg')

In [13]:
line[0]

'Функция 1 я Япунов '

In [15]:
img = cv2.imread(r'Скриншот 17-05-2024 232603.jpg')

In [19]:
pytesseract.image_to_string(img, lang='rus')

'Фамилия _ /& ‹’И(;/ иёв\n\n'